In [2]:
from py2neo import Node, Graph, Relationship

graph = Graph("bolt://neo4j:7687", name="neo4j", password="password")

In [113]:
query1 = '''
LOAD CSV WITH HEADERS FROM 
'file:///product_risk_2.csv' AS row
WITH row limit 200
WITH row,
(CASE 
    WHEN ToInteger(row.Missed_Payments) <5 THEN '0-5'
    WHEN ToInteger(row.Missed_Payments) <10 THEN '5-10'
    ELSE 'Above 10'
END) AS missed_pay_bin,

(CASE
    WHEN ToInteger(row.Primary_Credit_Score) < 500 THEN 'Below 500'
    WHEN ToInteger(row.Primary_Credit_Score) < 600 THEN '500-600'
    WHEN ToInteger(row.Primary_Credit_Score) < 700 THEN '600-700'
    ELSE 'Above 700'
END) AS credit_score_bin,

(CASE
    WHEN ToFloat(row.DTI) < 0.15 THEN 'Below 0.15'
    WHEN ToFloat(row.DTI) < 0.35 THEN '0.15-0.35'
    WHEN ToFloat(row.DTI) < 0.7 THEN '0.35-0.7'
    ELSE 'Above 0.7'
END) AS dti_bin
CREATE (c:customer {customer_id: (row.Customer_ID), age: ToFloat(row.Age), product_name: (row.Product_Name)})

MERGE (pd:probdefault {def : row.Probability_of_Default})
MERGE (c)-[:has_default]->(pd)

MERGE (pc:prod_code {product_name: row.Product_Name})
MERGE (c)-[:has_product]->(pc)

MERGE (mtb:missedpayment {missed_pay_bin : missed_pay_bin})
MERGE (c)-[:has_missed_payments {missed_payments: ToInteger(row.Missed_Payments)}]->(mtb)

MERGE (csb:creditscore {credit_score_bin : credit_score_bin})
MERGE (c)-[:has_credit_score {credit_score: ToInteger(row.Primary_Credit_Score)}]->(csb)

MERGE (dti:debttoincome {dti_bin : dti_bin})
MERGE (c)-[:has_debt_to_income_ratio {debt_to_income_ratio: ToFloat(row.DTI)}]->(dti)
'''

In [114]:
graph.run(query1).stats()

{'labels_added': 220,
 'relationships_created': 1000,
 'nodes_created': 220,
 'properties_set': 1220}

In [115]:
query2 = '''
        MATCH (pd:probdefault)<-[:has_default]-(c:customer)-[:has_product]->(pc)
        WITH pd.def as default, pc.product_name as product, count(DISTINCT c) as customer_count
        RETURN default, product, customer_count
'''

graph.run(query2).to_table()

default,product,customer_count
0,Personal Loan,25
0,Real Estate,38
0,Home Improvement,37
0,Mortgage,9
0,Car Loan,19
0,Line of credit,9
0,Line of Credit,18
0,Equipment Finance,32
1,Personal Loan,3
1,Equipment Finance,3


In [116]:
query3 = '''
        MATCH (pd:probdefault)<-[:has_default]-(c:customer)-[r:has_missed_payments]->(mtb:missedpayment)
        WITH pd.def as default, mtb.missed_pay_bin as missedbin, c.product_name as product, count(DISTINCT c) as customer_count
        RETURN default, missedbin, product, customer_count
'''

In [117]:
graph.run(query3).to_table()

default,missedbin,product,customer_count
0,Above 10,Car Loan,7
0,Above 10,Real Estate,10
0,Above 10,Personal Loan,5
1,Above 10,Personal Loan,3
0,Above 10,Home Improvement,16
0,Above 10,Equipment Finance,7
1,Above 10,Real Estate,5
1,Above 10,Home Improvement,2
0,Above 10,Line of Credit,6
1,Above 10,Equipment Finance,2


In [122]:
query4 = '''
        MATCH (pd:probdefault {def : '1'})<-[:has_default]-(c:customer)-[r1:has_missed_payments]->(mtb:missedpayment)
        MATCH (c:customer)-[r2:has_credit_score]->(csb:creditscore)
        MATCH (c:customer)-[r3:has_debt_to_income_ratio]->(dti:debttoincome)
        RETURN c.customer_id, c.product_name as product, r1.missed_payments as missed_payment,
        r2.credit_score as credit_score, round(r3.debt_to_income_ratio,3) as dti, c.age as age limit 10
'''
graph.run(query4).to_table()

c.customer_id,product,missed_payment,credit_score,dti,age
290758003894,Personal Loan,15,592,0.271,22.0
115232746270,Equipment Finance,9,517,0.39,66.0
806112590554,Personal Loan,10,475,0.349,20.0
159987371856,Home Improvement,13,850,0.37,51.0
283205437470,Personal Loan,12,580,0.34,28.0
623908435414,Real Estate,15,766,0.308,66.0
133483955087,Real Estate,14,697,0.272,55.0
496805003931,Equipment Finance,15,881,0.266,54.0
543160051584,Real Estate,14,721,0.412,35.0
866459345886,Real Estate,15,510,0.283,46.0


In [112]:
graph.run("MATCH (n) DETACH DELETE n").stats()

{'nodes_deleted': 220, 'relationships_deleted': 1000}